## _*Using Qiskit Aqua for clique problems*_

This Qiskit Aqua Optimization notebook demonstrates how to use the VQE quantum algorithm to compute the clique of a given graph. 

The problem is defined as follows. A clique in a graph $G$ is a complete subgraph of $G$. That is, it is a subset $K$ of the vertices such that every two vertices in $K$ are the two endpoints of an edge in $G$. A maximal clique is a clique to which no more vertices can be added. A maximum clique is a clique that includes the largest possible number of vertices. 

We will go through two examples to show:
1. How to run the optimization.
2. How how to run the optimization with the VQE.

Note that the solution may not be unique.

####  The problem and a brute-force method.

In [1]:
import numpy as np

from qiskit import BasicAer
from qiskit.optimization.applications.ising import clique
from qiskit.optimization.applications.ising.common import random_graph, sample_most_likely
from qiskit.aqua.algorithms import NumPyMinimumEigensolver

first, let us have a look at the graph, which is in the adjacent matrix form.

In [2]:
K = 3  # K means the size of the clique
np.random.seed(100)
num_nodes = 5
w = random_graph(num_nodes, edge_prob=0.8, weight_range=10)
print(w) 

[[ 0.  4.  5.  3. -5.]
 [ 4.  0.  7.  0.  6.]
 [ 5.  7.  0. -4.  0.]
 [ 3.  0. -4.  0.  8.]
 [-5.  6.  0.  8.  0.]]


Let us try a brute-force method. Basically, we exhaustively try all the binary assignments. In each binary assignment, the entry of a vertex is either 0 (meaning the vertex is not in the clique) or 1 (meaning the vertex is in the clique). We print the binary assignment that satisfies the definition of the clique (Note the size is specified as K).

In [3]:
def brute_force():
    # brute-force way: try every possible assignment!
    def bitfield(n, L):
        result = np.binary_repr(n, L)
        return [int(digit) for digit in result]

    L = num_nodes  # length of the bitstring that represents the assignment
    max = 2**L
    has_sol = False
    for i in range(max):
        cur = bitfield(i, L)
        cur_v = clique.satisfy_or_not(np.array(cur), w, K)
        if cur_v:
            has_sol = True
            break
    return has_sol, cur

has_sol, sol = brute_force()
if has_sol:
    print("Solution is ", sol)
else:
    print("No solution found for K=", K)

Solution is  [1, 0, 0, 1, 1]


In [4]:
qubit_op, offset = clique.get_operator(w, K)

#### Part I: Run the optimization using the programmatic approach

Here we directly construct the algorithm and then run() it to get the result.

In [5]:
# We will use the qubit_op and offset from above

algo = NumPyMinimumEigensolver(qubit_op)
result = algo.run()

x = sample_most_likely(result.eigenstate)
ising_sol = clique.get_graph_solution(x)
if clique.satisfy_or_not(ising_sol, w, K):
    print("Solution is", ising_sol)
else:
    print("No solution found for K=", K)     

Solution is [1 0 1 1 0]


#### Part II: Run the optimization with the VQE

We can create the objects directly ourselves too and run VQE for the result

In [6]:
from qiskit.aqua import aqua_globals
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal

aqua_globals.random_seed = 10598

optimizer = COBYLA()
var_form = TwoLocal(qubit_op.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
vqe = VQE(qubit_op, var_form, optimizer)

backend = BasicAer.get_backend('statevector_simulator')
result = vqe.run(backend)

x = sample_most_likely(result.eigenstate)
ising_sol = clique.get_graph_solution(x)

if clique.satisfy_or_not(ising_sol, w, K):
    print("Solution is", ising_sol)
else:
    print("No solution found for K=", K)

Solution is [1. 1. 1. 0. 0.]
